In [ ]:
#!pip install PyMySql
import pymysql
import pandas as pd
from mysql.connector import Error

In [ ]:
#Creating a DF from the API data
df = pd.read_csv('mempool_data_api.csv', index_col=0)
df.head()

In [ ]:
#Connect to MySQL database
def connect_to_db(host, user, password):
    try:
        conn = pymysql.connect(   
            host= host,
            user= user,
            password=password
        )
    except Error as e:
        print(e)
    else:
        print('Connected!')
    return conn

In [ ]:
#Creating the table
def create_table(curr):
    use_database = ("""USE xxxxx;""")
    create_table_command = ("""CREATE TABLE IF NOT EXISTS block_fees(
                        average_height INT NOT NULL PRIMARY KEY,
                        date DATETIME,
                        BTC_fees FLOAT,
                        USD_fees INT)""")
    #Executing commands using the cursor
    curr.execute(use_database)
    curr.execute(create_table_command)

In [ ]:
#Function for inserting the data into the DB
def insert_into_table(curr, average_height, date, BTC_fees, USD_fees):
    insert_into_mempool = (""" INSERT INTO block_fees(average_height, date, BTC_fees,
                            USD_fees) VALUES(%s, %s, %s, %s)""")
    row_to_insert = (average_height, date, BTC_fees, USD_fees)
    curr.execute(insert_into_mempool, row_to_insert)

In [ ]:
#Function for adding new rows to the db from the df
def append_from_df_to_db(curr, df):
    for i,row in df.iterrows():
        insert_into_table(curr, row['average_height'], row['date'], row['BTC_fees'], row['USD_fees'])

In [ ]:
#replacing old mempool data with new
def replace_with_new_data(curr, df):
    use_database = ("""USE xxxxxx;""")
    curr.execute(use_database)
    curr.execute("DROP TABLE block_fees")
    
    create_table(curr)
    append_from_df_to_db(curr, df)
    conn.commit()

In [ ]:
#Database credentials
host='xxxxxxxxx.rds.amazonaws.com'
user='xxxxxxxx'
password='xxxxxxx'
conn = None

#Connecting to the db
conn = connect_to_db(host, user, password)
#Allows to execute SQL commands
curr = conn.cursor()

In [ ]:
#create mempool table for the FIRST time
create_table(curr)

In [ ]:
#add new blocks to the table
append_from_df_to_db(curr, df)
conn.commit()

In [ ]:
#If old mempool data exists in the database and you need to
#replace it with the current data
replace_with_new_data(curr, df)

In [ ]:
#Checking if the data is successfully added to the db
curr.execute("SELECT BTC_fees FROM block_fees LIMIT 5")
print(curr.fetchall())